In [1]:
# ###############################################################################################
# ANN, MLK and SVM machine learing models are applied to predict the future stock price movement.
# output window size: next 1, 3, 7,10, 14, 21,28,45 and 60 days for close prices.
#features used: full features(36), selected (11) features by Extremely Randomized Trees 
#################################################################################################
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
# Import the necessary libraries first
# Import required libraries
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
from tensorflow.keras.utils import to_categorical


In [2]:
#read spy&500 technical indicators data and add a prefic s_ to each column name.
file_path="./retrieveData/data"
spy = = pd.read_csv(os.path.join(file_path,"spy.csv"))
spy.head()
spyi=spy[['date','adj_close', 'sma', 'rsi', 'slowd',
       'slowk', 'roc', 'willr', 'std', 'mom', 'adx', 'obv', 'atr', 'cci',
       'macd', 'trix']]
spyidx=spyi.add_prefix('s_')
spyidx.head()

,s_date,s_adj_close,s_sma,s_rsi,s_slowd,s_slowk,s_roc,s_willr,s_std,s_mom,s_adx,s_obv,s_atr,s_cci,s_macd,s_trix
0,2010-07-09,88.2266,106.3943,50.6201,73.0817,98.4491,-3.3742,-43.4134,4.18525,-3.77,31.8911,2.712688e+09,2.1703,31.2417,-1.4158,-0.2092
1,2010-07-12,88.2838,106.1529,50.8352,90.3016,98.1965,-3.0338,-35.9331,3.71105,-3.38,30.3314,2.843649e+09,2.0931,49.0562,-1.1685,-0.1989
2,2010-07-13,89.6158,106.1593,55.6767,97.8217,96.8195,0.0821,-4.7991,3.72450,0.09,28.4013,3.056193e+09,2.0907,106.0754,-0.8314,-0.1811
3,2010-07-14,89.6077,106.1893,55.6405,96.1264,93.3633,0.3845,-4.9107,3.78325,0.42,26.5727,2.871863e+09,2.0285,103.6809,-0.5586,-0.1588
4,2010-07-15,89.6322,106.3507,55.7335,93.4406,90.1390,2.1039,-4.5759,3.99615,2.26,24.8394,3.103545e+09,2.0186,89.5847,-0.3362,-0.1343


In [124]:
#read a company daily stock prices and technical indicators
ticker='JPM'
file_path="./retrieveData/data"
data = pd.read_csv(os.path.join(file_path,f"{ticker}.csv"))
# amzn = pd.read_csv("newAMZN.csv")
data.head()

,Unnamed: 0,ticker,date,open,high,low,close,volume,dividend,split,...,std,mom,adx,obv,atr,cci,macd,trix,next_close,direction
0,0,JPM,2010-07-09,37.75,38.89,37.68,38.85,30422300,0.0,1.0,...,1.71130,-0.33,23.9306,-1.580404e+09,1.3432,47.4085,-0.4411,-0.2134,39.19,1.0
1,1,JPM,2010-07-12,38.86,39.34,38.60,39.19,24998900,0.0,1.0,...,1.74760,0.32,22.3151,-1.555405e+09,1.3001,87.6849,-0.2933,-0.1893,40.48,1.0
2,2,JPM,2010-07-13,39.81,40.69,39.74,40.48,48564800,0.0,1.0,...,2.02385,2.15,21.9751,-1.506840e+09,1.3144,152.1621,-0.0713,-0.1523,40.35,-1.0
3,3,JPM,2010-07-14,40.15,40.38,39.84,40.35,37320600,0.0,1.0,...,2.20085,1.46,21.6594,-1.544161e+09,1.2662,126.9225,0.0932,-0.1092,40.46,1.0
4,4,JPM,2010-07-15,40.72,40.84,39.31,40.46,81876200,0.0,1.0,...,2.38480,2.43,20.7809,-1.462284e+09,1.2851,105.0773,0.2297,-0.0631,39.00,-1.0


In [125]:
# merge two datasets
mixDt=pd.concat([data,spyidx],axis=1)
mixDt['day']=mixDt.index
mixDt=mixDt.iloc[:-1]
mixDt.columns

Index(['Unnamed: 0', 'ticker', 'date', 'open', 'high', 'low', 'close',
       'volume', 'dividend', 'split', 'adj_close', 'sma', 'rsi', 'slowd',
       'slowk', 'roc', 'willr', 'std', 'mom', 'adx', 'obv', 'atr', 'cci',
       'macd', 'trix', 'next_close', 'direction', 's_date', 's_adj_close',
       's_sma', 's_rsi', 's_slowd', 's_slowk', 's_roc', 's_willr', 's_std',
       's_mom', 's_adx', 's_obv', 's_atr', 's_cci', 's_macd', 's_trix', 'day'],
      dtype='object')

In [126]:
#create future close price for next 3,7,10,14,21,28,45 and 60 days
mixDt['next3']=mixDt['close'].shift(-3)
mixDt['next7']=mixDt['close'].shift(-7)
mixDt['next10']=mixDt['close'].shift(-10)
mixDt['next14']=mixDt['close'].shift(-14)
mixDt['next21']=mixDt['close'].shift(-21)
mixDt['next28']=mixDt['close'].shift(-28)
mixDt['next45']=mixDt['close'].shift(-45)
mixDt['next60']=mixDt['close'].shift(-60)
mixDt=mixDt.iloc[:-60]

In [127]:
mixDt.loc[(mixDt['next3']-mixDt['close']) >0, 'direction3'] = '1' 
mixDt.loc[(mixDt['next3']-mixDt['close']) <=0, 'direction3'] = '0' 
mixDt['direction3']=mixDt['direction3'].astype(int)

In [128]:
mixDt.loc[(mixDt['next7']-mixDt['close']) >0, 'direction7'] = '1' 
mixDt.loc[(mixDt['next7']-mixDt['close']) <=0, 'direction7'] = '0' 
mixDt['direction7']=mixDt['direction7'].astype(int)

In [129]:
mixDt.loc[(mixDt['next14']-mixDt['close']) >0, 'direction14'] = '1' 
mixDt.loc[(mixDt['next14']-mixDt['close']) <=0, 'direction14'] = '0' 
mixDt['direction14']=mixDt['direction14'].astype(int)

In [130]:
mixDt.loc[(mixDt['next21']-mixDt['close']) >0, 'direction21'] = '1' 
mixDt.loc[(mixDt['next21']-mixDt['close']) <=0, 'direction21'] = '0' 
mixDt['direction21']=mixDt['direction21'].astype(int)

In [131]:
mixDt.loc[(mixDt['next28']-mixDt['close']) >0, 'direction28'] = '1' 
mixDt.loc[(mixDt['next28']-mixDt['close']) <=0, 'direction28'] = '0' 
mixDt['direction28']=mixDt['direction28'].astype(int)

In [132]:
mixDt.loc[(mixDt['next45']-mixDt['close']) >0, 'direction45'] = '1' 
mixDt.loc[(mixDt['next45']-mixDt['close']) <=0, 'direction45'] = '0' 
mixDt['direction45']=mixDt['direction45'].astype(int)

In [133]:
mixDt.loc[(mixDt['next10']-mixDt['close']) >0, 'direction10'] = '1' 
mixDt.loc[(mixDt['next10']-mixDt['close']) <=0, 'direction10'] = '0' 
mixDt['direction10']=mixDt['direction10'].astype(int)

In [134]:
mixDt.loc[(mixDt['next60']-mixDt['close']) >0, 'direction60'] = '1' 
mixDt.loc[(mixDt['next60']-mixDt['close']) <=0, 'direction60'] = '0' 
mixDt['direction60']=mixDt['direction60'].astype(int)

In [135]:
mixDt.loc[(mixDt['direction']) ==-1, 'direction'] = '0' 
mixDt['direction']=mixDt['direction'].astype(int)


In [136]:
#create list of features, and output target 
target_column = ['direction7'] 
predictors = ['open', 'high', 'low', 'close',
       'volume', 'adj_close', 'sma', 'rsi', 'slowd',
       'slowk', 'roc', 'willr', 'std', 'mom', 'adx', 'obv', 'atr', 'cci',
       'macd', 'trix', 's_adj_close',
       's_sma', 's_rsi', 's_slowd', 's_slowk', 's_roc', 's_willr', 's_std',
       's_mom', 's_adx', 's_obv', 's_atr', 's_cci', 's_macd', 's_trix', 'day']

In [137]:
#create list of selected 11 features
predictors_selected = ['low', 'close','adx', 'obv', 'atr', 'trix', 's_adj_close','s_std',
        's_adx',  's_atr', 'day']

In [138]:
#splite train, test dataset
X = mixDt[predictors].values
y = mixDt[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(1723, 36)
(739, 36)


In [139]:
#create scaler for data normalization 

#X_scaler = StandardScaler().fit(X_train)

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [142]:
 # One-hot encoding for output Y 
num_classes = 2
y_train_categorical = to_categorical(y_train,num_classes)
y_test_categorical = to_categorical(y_test,num_classes)
y_train_categorical

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [143]:
# MLP model from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(12,16,16), activation='relu', solver='adam', max_iter=2000)
mlp.fit(X_train_scaled,y_train_categorical)

predict_train = mlp.predict(X_train_scaled)
predict_test = mlp.predict(X_test_scaled)

In [144]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
#print(confusion_matrix(y_train_categorical,predict_train))
print(classification_report(y_train_categorical,predict_train))

              precision    recall  f1-score   support

           0       0.62      0.72      0.67       714
           1       0.77      0.69      0.73      1009

   micro avg       0.70      0.70      0.70      1723
   macro avg       0.70      0.71      0.70      1723
weighted avg       0.71      0.70      0.70      1723
 samples avg       0.70      0.70      0.70      1723



/Users/jadetao/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [145]:
print(classification_report(y_test_categorical,predict_test))

              precision    recall  f1-score   support

           0       0.57      0.62      0.59       325
           1       0.68      0.64      0.66       414

   micro avg       0.63      0.63      0.63       739
   macro avg       0.63      0.63      0.63       739
weighted avg       0.63      0.63      0.63       739
 samples avg       0.63      0.63      0.63       739



In [146]:
print (accuracy_score(y_test_categorical,predict_test))

0.625169147496617


In [147]:
#ANN model
from tensorflow.keras.models import Sequential

model = Sequential()

In [148]:
from tensorflow.keras.layers import Dense
number_inputs =36
number_hidden_nodes =36 
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [149]:
model.add(Dense(20, activation='relu'))

In [150]:
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax'))

In [151]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 36)                1332      
_________________________________________________________________
dense_13 (Dense)             (None, 20)                740       
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 42        
Total params: 2,114
Trainable params: 2,114
Non-trainable params: 0
_________________________________________________________________


In [152]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [153]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Train on 1723 samples
Epoch 1/1000
1723/1723 - 1s - loss: 0.6860 - accuracy: 0.5647
Epoch 2/1000
1723/1723 - 0s - loss: 0.6795 - accuracy: 0.5810
Epoch 3/1000
1723/1723 - 0s - loss: 0.6736 - accuracy: 0.5850
Epoch 4/1000
1723/1723 - 0s - loss: 0.6704 - accuracy: 0.5856
Epoch 5/1000
1723/1723 - 0s - loss: 0.6703 - accuracy: 0.5827
Epoch 6/1000
1723/1723 - 0s - loss: 0.6663 - accuracy: 0.5873
Epoch 7/1000
1723/1723 - 0s - loss: 0.6654 - accuracy: 0.5891
Epoch 8/1000
1723/1723 - 0s - loss: 0.6656 - accuracy: 0.5908
Epoch 9/1000
1723/1723 - 0s - loss: 0.6639 - accuracy: 0.5984
Epoch 10/1000
1723/1723 - 0s - loss: 0.6612 - accuracy: 0.5949
Epoch 11/1000
1723/1723 - 0s - loss: 0.6615 - accuracy: 0.5972
Epoch 12/1000
1723/1723 - 0s - loss: 0.6601 - accuracy: 0.6013
Epoch 13/1000
1723/1723 - 0s - loss: 0.6601 - accuracy: 0.6071
Epoch 14/1000
1723/1723 - 0s - loss: 0.6585 - accuracy: 0.6001
Epoch 15/1000
1723/1723 - 0s - loss: 0.6591 - accuracy: 0.6065
Epoch 16/1000
1723/1723 - 0s - loss: 0.654

Epoch 131/1000
1723/1723 - 0s - loss: 0.4941 - accuracy: 0.7574
Epoch 132/1000
1723/1723 - 0s - loss: 0.4926 - accuracy: 0.7504
Epoch 133/1000
1723/1723 - 0s - loss: 0.4848 - accuracy: 0.7615
Epoch 134/1000
1723/1723 - 0s - loss: 0.4887 - accuracy: 0.7557
Epoch 135/1000
1723/1723 - 0s - loss: 0.4899 - accuracy: 0.7678
Epoch 136/1000
1723/1723 - 0s - loss: 0.4862 - accuracy: 0.7626
Epoch 137/1000
1723/1723 - 0s - loss: 0.4840 - accuracy: 0.7603
Epoch 138/1000
1723/1723 - 0s - loss: 0.4823 - accuracy: 0.7678
Epoch 139/1000
1723/1723 - 0s - loss: 0.4824 - accuracy: 0.7620
Epoch 140/1000
1723/1723 - 0s - loss: 0.4810 - accuracy: 0.7597
Epoch 141/1000
1723/1723 - 0s - loss: 0.4824 - accuracy: 0.7673
Epoch 142/1000
1723/1723 - 0s - loss: 0.4783 - accuracy: 0.7690
Epoch 143/1000
1723/1723 - 0s - loss: 0.4834 - accuracy: 0.7684
Epoch 144/1000
1723/1723 - 0s - loss: 0.4757 - accuracy: 0.7649
Epoch 145/1000
1723/1723 - 0s - loss: 0.4731 - accuracy: 0.7696
Epoch 146/1000
1723/1723 - 0s - loss: 0.

1723/1723 - 0s - loss: 0.3910 - accuracy: 0.8259
Epoch 260/1000
1723/1723 - 0s - loss: 0.3872 - accuracy: 0.8299
Epoch 261/1000
1723/1723 - 0s - loss: 0.3823 - accuracy: 0.8323
Epoch 262/1000
1723/1723 - 0s - loss: 0.3803 - accuracy: 0.8288
Epoch 263/1000
1723/1723 - 0s - loss: 0.3854 - accuracy: 0.8178
Epoch 264/1000
1723/1723 - 0s - loss: 0.3835 - accuracy: 0.8253
Epoch 265/1000
1723/1723 - 0s - loss: 0.3837 - accuracy: 0.8201
Epoch 266/1000
1723/1723 - 0s - loss: 0.3752 - accuracy: 0.8282
Epoch 267/1000
1723/1723 - 0s - loss: 0.3862 - accuracy: 0.8259
Epoch 268/1000
1723/1723 - 0s - loss: 0.3748 - accuracy: 0.8299
Epoch 269/1000
1723/1723 - 0s - loss: 0.3752 - accuracy: 0.8363
Epoch 270/1000
1723/1723 - 0s - loss: 0.3794 - accuracy: 0.8299
Epoch 271/1000
1723/1723 - 0s - loss: 0.3804 - accuracy: 0.8224
Epoch 272/1000
1723/1723 - 0s - loss: 0.3849 - accuracy: 0.8270
Epoch 273/1000
1723/1723 - 0s - loss: 0.3770 - accuracy: 0.8334
Epoch 274/1000
1723/1723 - 0s - loss: 0.3834 - accuracy

Epoch 388/1000
1723/1723 - 0s - loss: 0.3239 - accuracy: 0.8584
Epoch 389/1000
1723/1723 - 0s - loss: 0.3241 - accuracy: 0.8561
Epoch 390/1000
1723/1723 - 0s - loss: 0.3128 - accuracy: 0.8706
Epoch 391/1000
1723/1723 - 0s - loss: 0.3267 - accuracy: 0.8636
Epoch 392/1000
1723/1723 - 0s - loss: 0.3205 - accuracy: 0.8578
Epoch 393/1000
1723/1723 - 0s - loss: 0.3176 - accuracy: 0.8624
Epoch 394/1000
1723/1723 - 0s - loss: 0.3232 - accuracy: 0.8671
Epoch 395/1000
1723/1723 - 0s - loss: 0.3231 - accuracy: 0.8624
Epoch 396/1000
1723/1723 - 0s - loss: 0.3111 - accuracy: 0.8712
Epoch 397/1000
1723/1723 - 0s - loss: 0.3141 - accuracy: 0.8665
Epoch 398/1000
1723/1723 - 0s - loss: 0.3149 - accuracy: 0.8712
Epoch 399/1000
1723/1723 - 0s - loss: 0.3197 - accuracy: 0.8595
Epoch 400/1000
1723/1723 - 0s - loss: 0.3269 - accuracy: 0.8566
Epoch 401/1000
1723/1723 - 0s - loss: 0.3172 - accuracy: 0.8624
Epoch 402/1000
1723/1723 - 0s - loss: 0.3105 - accuracy: 0.8706
Epoch 403/1000
1723/1723 - 0s - loss: 0.

1723/1723 - 0s - loss: 0.2724 - accuracy: 0.8874
Epoch 517/1000
1723/1723 - 0s - loss: 0.2845 - accuracy: 0.8851
Epoch 518/1000
1723/1723 - 0s - loss: 0.2757 - accuracy: 0.8851
Epoch 519/1000
1723/1723 - 0s - loss: 0.2804 - accuracy: 0.8804
Epoch 520/1000
1723/1723 - 0s - loss: 0.2828 - accuracy: 0.8886
Epoch 521/1000
1723/1723 - 0s - loss: 0.2708 - accuracy: 0.8891
Epoch 522/1000
1723/1723 - 0s - loss: 0.2820 - accuracy: 0.8926
Epoch 523/1000
1723/1723 - 0s - loss: 0.2762 - accuracy: 0.8932
Epoch 524/1000
1723/1723 - 0s - loss: 0.2743 - accuracy: 0.8932
Epoch 525/1000
1723/1723 - 0s - loss: 0.2618 - accuracy: 0.8973
Epoch 526/1000
1723/1723 - 0s - loss: 0.2714 - accuracy: 0.8886
Epoch 527/1000
1723/1723 - 0s - loss: 0.2690 - accuracy: 0.8886
Epoch 528/1000
1723/1723 - 0s - loss: 0.2738 - accuracy: 0.8874
Epoch 529/1000
1723/1723 - 0s - loss: 0.2706 - accuracy: 0.8909
Epoch 530/1000
1723/1723 - 0s - loss: 0.2688 - accuracy: 0.8909
Epoch 531/1000
1723/1723 - 0s - loss: 0.2632 - accuracy

Epoch 645/1000
1723/1723 - 0s - loss: 0.2265 - accuracy: 0.9129
Epoch 646/1000
1723/1723 - 0s - loss: 0.2316 - accuracy: 0.9083
Epoch 647/1000
1723/1723 - 0s - loss: 0.2443 - accuracy: 0.8967
Epoch 648/1000
1723/1723 - 0s - loss: 0.2313 - accuracy: 0.9095
Epoch 649/1000
1723/1723 - 0s - loss: 0.2330 - accuracy: 0.9135
Epoch 650/1000
1723/1723 - 0s - loss: 0.2546 - accuracy: 0.8944
Epoch 651/1000
1723/1723 - 0s - loss: 0.2417 - accuracy: 0.9025
Epoch 652/1000
1723/1723 - 0s - loss: 0.2323 - accuracy: 0.9089
Epoch 653/1000
1723/1723 - 0s - loss: 0.2342 - accuracy: 0.9054
Epoch 654/1000
1723/1723 - 0s - loss: 0.2278 - accuracy: 0.9118
Epoch 655/1000
1723/1723 - 0s - loss: 0.2404 - accuracy: 0.9037
Epoch 656/1000
1723/1723 - 0s - loss: 0.2465 - accuracy: 0.9002
Epoch 657/1000
1723/1723 - 0s - loss: 0.2279 - accuracy: 0.9083
Epoch 658/1000
1723/1723 - 0s - loss: 0.2632 - accuracy: 0.8845
Epoch 659/1000
1723/1723 - 0s - loss: 0.2407 - accuracy: 0.9042
Epoch 660/1000
1723/1723 - 0s - loss: 0.

1723/1723 - 0s - loss: 0.2123 - accuracy: 0.9141
Epoch 774/1000
1723/1723 - 0s - loss: 0.2054 - accuracy: 0.9211
Epoch 775/1000
1723/1723 - 0s - loss: 0.2061 - accuracy: 0.9193
Epoch 776/1000
1723/1723 - 0s - loss: 0.2021 - accuracy: 0.9193
Epoch 777/1000
1723/1723 - 0s - loss: 0.1942 - accuracy: 0.9292
Epoch 778/1000
1723/1723 - 0s - loss: 0.1954 - accuracy: 0.9228
Epoch 779/1000
1723/1723 - 0s - loss: 0.1938 - accuracy: 0.9263
Epoch 780/1000
1723/1723 - 0s - loss: 0.2035 - accuracy: 0.9193
Epoch 781/1000
1723/1723 - 0s - loss: 0.2226 - accuracy: 0.9141
Epoch 782/1000
1723/1723 - 0s - loss: 0.2149 - accuracy: 0.9158
Epoch 783/1000
1723/1723 - 0s - loss: 0.2114 - accuracy: 0.9158
Epoch 784/1000
1723/1723 - 0s - loss: 0.1968 - accuracy: 0.9257
Epoch 785/1000
1723/1723 - 0s - loss: 0.2013 - accuracy: 0.9182
Epoch 786/1000
1723/1723 - 0s - loss: 0.2051 - accuracy: 0.9147
Epoch 787/1000
1723/1723 - 0s - loss: 0.1964 - accuracy: 0.9298
Epoch 788/1000
1723/1723 - 0s - loss: 0.1959 - accuracy

Epoch 902/1000
1723/1723 - 0s - loss: 0.1764 - accuracy: 0.9327
Epoch 903/1000
1723/1723 - 0s - loss: 0.1878 - accuracy: 0.9304
Epoch 904/1000
1723/1723 - 0s - loss: 0.1841 - accuracy: 0.9315
Epoch 905/1000
1723/1723 - 0s - loss: 0.1833 - accuracy: 0.9269
Epoch 906/1000
1723/1723 - 0s - loss: 0.1780 - accuracy: 0.9309
Epoch 907/1000
1723/1723 - 0s - loss: 0.1743 - accuracy: 0.9333
Epoch 908/1000
1723/1723 - 0s - loss: 0.1856 - accuracy: 0.9298
Epoch 909/1000
1723/1723 - 0s - loss: 0.2179 - accuracy: 0.9048
Epoch 910/1000
1723/1723 - 0s - loss: 0.1768 - accuracy: 0.9327
Epoch 911/1000
1723/1723 - 0s - loss: 0.1729 - accuracy: 0.9309
Epoch 912/1000
1723/1723 - 0s - loss: 0.1733 - accuracy: 0.9344
Epoch 913/1000
1723/1723 - 0s - loss: 0.1966 - accuracy: 0.9164
Epoch 914/1000
1723/1723 - 0s - loss: 0.1728 - accuracy: 0.9373
Epoch 915/1000
1723/1723 - 0s - loss: 0.1775 - accuracy: 0.9338
Epoch 916/1000
1723/1723 - 0s - loss: 0.1771 - accuracy: 0.9327
Epoch 917/1000
1723/1723 - 0s - loss: 0.

In [154]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

739/1 - 0s - loss: 0.4353 - accuracy: 0.7659
Loss: 0.8677592050546908, Accuracy: 0.7658998370170593


In [155]:
 # Support vector machine classifier 
target_names = ["negative", "positive"]


In [156]:
X = mixDt[predictors].values
y = mixDt[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(1723, 36)
(739, 36)


In [158]:
from sklearn.svm import SVC 
model = SVC(kernel='rbf',C=1e3, gamma=0.1,max_iter=5000)
model.fit(X_train_scaled, y_train)

/Users/jadetao/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jadetao/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1000.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=5000, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [160]:
 # Model Accuracy
print('Test Acc: %.3f' % model.score(X_test_scaled, y_test))

Test Acc: 0.555
